## 朴素贝叶斯(Naive Bayes)

### 介绍

- 基于贝叶斯计算预测函数时, 都要基于此公式: 
$$ P(y|x) = \frac{P(x|y)P(y)}{P(x)}$$
$$ P(\text{类别} \mid \text{特征})=\frac{P(\text{特征} \mid \text{类别}) \times P(\text{类别})}{P(\text{特征})} $$
- $ P(y|x)$ 是给定参数x后y的概率, 也就是对应了预测函数, 也是所谓的后验概率
- $ P(x|y)$ 是给定结果y后x的概率, 是根据结果推测最可能的参数, 也就是似然性
- $ P(y)$ 是结果本身的为某值的概率, 是先验概率
- $ P(x)$ 是参数发生的概率, 是先验概率, 一般可视为常量

- 朴素贝叶斯的假设 参数条件是相互独立的, 即: $ x_i x_j | y(i \neq j)$, 目的是为了简化运算, 每个参数条件都可以单独计算分布, 最后合并

- 因此, 模型有: $$ P(y|x) = \prod_{i=1}^{N}(P(x_i|y)P(y))$$


#### 贝叶斯估计

- 在做极大似然估计时，若类别中缺少一些特征，则就会出现概率值为 0 的情况。会影响后验概率的计算结果，使得分类产生偏差。解决这一问题最好的方法就是采用贝叶斯估计
$$ P(y_{i}=c_{k})=\frac{\sum_{i=1}^{N}I(y_{i}=c_{k})+\lambda}{N+k\lambda}$$

- 当 $\lambda=0$ 时就是极大似然估计
- 当 $\lambda=1$ 时称为拉普拉斯平滑, 每一个分类都会加上1, 分母k为总分类数, 分类指某一个维度能有多少个项

### 代码

In [12]:
import pandas as pd
import numpy as np



def create_data():
    # 生成示例数据
    data = {
        "x": ["r","g","r","b","g","g","r","r","b","g","g","r","b","b","g",],
        "y": ["m","s","l","s","m","s","m","s","m","l","l","s","m","m","l",],
        "labels": ["A","A","A","A","A","A","A","A","B","B","B","B","B","B","B",],
    }
    data = pd.DataFrame(data, columns=["labels", "x", "y"])
    return data

data = create_data()


def get_P_labels(labels):
    # P(\text{种类}) 先验概率计算
    labels = list(labels)  # 转换为 list 类型
    P_label = {}  # 设置空字典用于存入 label 的概率
    for label in labels:
        # 统计 label 标签在标签集中出现的次数再除以总长度
        P_label[label] = labels.count(label) / float(
            len(labels)
        )  # p = count(y) / count(Y)
    return P_label


P_labels = get_P_labels(data["labels"])
print(P_labels)

# 将 data 中的属性切割出来，即 x 和 y 属性
train_data = np.array(data.iloc[:, 1:])

labels = data["labels"]
label_index = []
# 遍历所有的标签，这里就是将标签为 A 和 B 的数据集分开，label_index 中存的是该数据的下标
for y in P_labels.keys():
    temp_index = []
    # enumerate 函数返回 Series 类型数的索引和值，其中 i 为索引，label 为值
    for i, label in enumerate(labels):
        if label == y:
            temp_index.append(i)
        else:
            pass
    label_index.append(temp_index)
# 遍历 train_data 中的第一列数据，提取出里面内容为r的数据
x_index = [
    i for i, feature in enumerate(train_data[:, 0]) if feature == "r"
]  # 效果等同于求类别索引中 for 循环
print(label_index)


def get_P_fea_lab(P_label, features, data):
    # P(\text{特征}∣种类) 先验概率计算
    # 该函数就是求 种类为 P_label 条件下特征为 features 的概率
    P_fea_lab = {}
    train_data = data.iloc[:, 1:]
    train_data = np.array(train_data)
    labels = data["labels"]
    # 遍历所有的标签
    for each_label in P_label.keys():
        # 上面代码的另一种写法，这里就是将标签为 A 和 B 的数据集分开，label_index 中存的是该数据的下标
        label_index = [i for i, label in enumerate(labels) if label == each_label]

        # 遍历该属性下的所有取值
        # 求出每种标签下，该属性取每种值的概率
        for j in range(len(features)):
            # 筛选出该属性下属性值为 features[j] 的数据
            feature_index = [
                i
                for i, feature in enumerate(train_data[:, j])
                if feature == features[j]
            ]

            # set(x_index)&set(y_index) 取交集，得到标签值为 each_label,属性值为 features[j] 的数据集合
            fea_lab_count = len(set(feature_index) & set(label_index))
            key = str(features[j]) + "|" + str(each_label)  # 拼接字符串

            # 计算先验概率
            # 计算 labels 为 each_label下，featurs 为 features[j] 的概率值
            P_fea_lab[key] = fea_lab_count / float(len(label_index))
    return P_fea_lab


features = ["r", "m"]
get_P_fea_lab(P_labels, features, data)

{'A': 0.5333333333333333, 'B': 0.4666666666666667}
[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14]]


{'r|A': 0.5,
 'm|A': 0.375,
 'r|B': 0.14285714285714285,
 'm|B': 0.42857142857142855}

In [13]:
def classify(data, features):
    # 朴素贝叶斯分类器
    # 求 labels 中每个 label 的先验概率
    labels = data["labels"]
    # 这里也就是求 P（B），P_label 为一个字典，存的是每个 B 对应的 P(B)
    P_label = get_P_labels(labels)
    P_fea_lab = get_P_fea_lab(P_label, features, data)  # 这里是在求 P（A|B）

    P = {}
    P_show = {}  # 后验概率
    for each_label in P_label:
        P[each_label] = P_label[each_label]
        # 遍历每个标签下的每种属性
        for each_feature in features:
            # 拼接字符串为 P(B/A) 用于字典的键值
            key = str(each_label) + "|" + str(features)
            # 计算 P(B/A) = P(B) * P(A/B) 因为所有的后验概率，分母相同。因此，在计算时可以忽略掉。
            P_show[key] = (
                P[each_label] * P_fea_lab[str(each_feature) + "|" + str(each_label)]
            )
            # 把刚才算的概率放到 P 列表里面，这个 P 列表的键值变成了标签。
            # 这样做的目的，其实是为了在后面取最大时，取出就是标签，而不是 标签|特征
            P[each_label] = (
                P[each_label] * P_fea_lab[str(each_feature) + "|" + str(each_label)]
            )
    # 输出 P_show 和 P 观察，发现他们的概率值没有变，只是字典的 key 值变了
    print(P_show)
    print(P)
    features_label = max(P, key=P.get)  # 概率最大值对应的类别
    return features_label

classify(data, ["r", "m"])

{"A|['r', 'm']": 0.1, "B|['r', 'm']": 0.02857142857142857}
{'A': 0.1, 'B': 0.02857142857142857}


'A'

#### 邮件过滤

In [22]:
import re
import jieba
data = pd.read_table('E:/vscode/file/trec06c/full/index', header=None,
                     encoding='gb2312', delim_whitespace=True)
df = data.replace(['spam', 'ham'], [0, 1])  # 0 替代 spam，1 替代 ham
df = df.replace(regex=["\.."], value='E:/vscode/file/trec06c')  # 替换掉文件路径
df = df.sample(len(df), random_state=1, )[:10000]  # 打乱样本并取前 1 万条数据
df.groupby(0).count()  # 统计样本


def load_stopwords(file_path):
    # 加载停用词
    with open(file_path, 'r', encoding="utf-8") as f:
        stopwords = [line.strip('\n') for line in f.readlines()]
    return stopwords

stopwords = load_stopwords('E:/vscode/file/trec06c/stopwords.txt')
stopwords

def clean_str(line):
    # 清理邮件，替换不需要的字符串
    line.strip('\n')
    line = re.sub(r"[^\u4e00-\u9fff]", "", line)
    line = re.sub("[0-9a-zA-Z\-\s+\.\!\/_,$%^*\(\)\+(+\"\')]+|[+——！，。？、~@#￥%……&*（）<>\[\]:：★◆【】《》;；=?？]+", "", line)
    return line.strip()


def process(file_path, test_mode=False):
    # 清洗一封邮件
    '''
    - file_path: 文本文件路径
    - test_mode: 测试模式，后文我们会将一个字符串写入文件(utf-8 编码)，而训练文件以 GBK 编码，
                 如果自己实现分类，请注意编码格式，通常为 utf-8
    - return: words, 处理、分词之后的有效词语
    '''
    words = []
    with open(file_path, 'rb') as f:
        for line in f.readlines():
            if not test_mode:
                line = line.strip().decode("gbk", 'ignore')
            else:
                line = line.strip().decode("utf-8", 'ignore')
            line = clean_str(line)
            if len(line) == 0:
                continue
            seg_list = list(jieba.cut(line, cut_all=False))
            for x in seg_list:
                if len(x) <= 1:
                    continue
                if x in stopwords:
                    continue
                words.append(x)
    return words

words = process('E:/vscode/file/trec06c/data/000/000')
" ".join(words)

C:\Users\issuser\AppData\Local\Temp\ipykernel_1080\3783788360.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_table('E:/vscode/file/trec06c/full/index', header=None,
C:\Users\issuser\AppData\Local\Temp\ipykernel_1080\3783788360.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = data.replace(['spam', 'ham'], [0, 1])  # 0 替代 spam，1 替代 ham


'上海 培训 课程 财务 纠淼 沙盘 模拟 财务 课程 背景 一位 管理 技术人员 懂得 技术 角度 衡量 合算 方案 也许 却是 财务 陷阱 表面 赢利 亏损 使经 营者 接受 技术手段 财务 运作 相结合 每位 管理 技术人员 老板 角度 思考 规避 财务 陷阱 管理决策 目标 一致性 课程 沙盘 模拟 案例 分析 企业 管理 技术人员 财务管理 知识 利用 财务 信息 改进 管理决策 管理 效益 最大化 学习 课程 会计 财务管理 提高 日常 管理 活动 财务 可行性 业绩 评价 方法 评估 业绩 实施 科学 业绩考核 合乎 财务 墓芾 老板 思维 同步 分析 关键 业绩 指标 战略规划 预算 企业 管理 重心 管理 系统性 课程 大纲 财务 工作 内容 作用 财务会计 财务 专家 思维 模式 财务 工作 内容 管理者 利用 财务 管理 决策 阅读 分析 财务报表 会计报表 损益表 阅读 分析 资产 负债表 阅读 分析 资金 流量 现金流量 阅读 分析 会计报表 之间 关系 会计报表 读懂 企业 状况 案例 分析 报表 判断 企业 业绩 水平 财务 手段 成本 控制 产品成本 概念 本浚利 分析 标准 成本 制度 成本 控制 作用 目标 成本法 控制 产品成本 保证 利润 水平 作业 成本法 管理 分析 实施 精细 成本 管理 沉没 成本 机会成本 正确 决策 改善 采购 生产 环节 运作 改良 企业 整体 财务状况 综合 案例 分析 财务 尚械 墓芾 醴桨 管理 技术 方案 可行性 分析 产品开发 财务 可行性 分析 产品 增产 减产 财务 可行性 分析 生产 设备 改造 更新 决策分析 投资 项目 现金流 分析 投资 项目 评价 方法 现值 法分析 资金 时间 价值 分析 综合 案例 演练 公司 费用 控制 公司 费用 控制 费用 方法 影响 费用 因素 分析 成本 中心 费用 控制 利润 中心 业绩考核 投资 中心 业绩 评价 利用 财务 数据分析 改善 绩效 公司财务 分析 核心 思路 关键 财务指标 解析 盈利 能力 分析 资产 回报率 股东权益 回报率 资产 流动 速率 风险 指数 分析 流动比率 负债 权益 比率 营运 偿债 能力 财务报表 综合 解读 综合 财务 信息 透视 公司 运作 水平 案例 分析 上市公司 财务状况 分析 评价 企业 运

In [23]:
from tqdm import tqdm

tqdm.pandas()  # 使用 tqdm 显示进度
# 将 apply 函数替换为 progress_apply 以使用 tqdm 显示处理进度
df["words"] = df[1].progress_apply(process)
df.head()

100%|██████████| 10000/10000 [00:35<00:00, 283.10it/s]


,0,1,words
37029,1,E:/vscode/file/trec06c/data/123/129,"[恋爱, 第三次, 告诉, 再见面, 时间, 我要, 考研, 考到, 北京, 是否是, 喜欢..."
2257,0,E:/vscode/file/trec06c/data/007/157,"[欣欣, 签约, 推出, 中国, 第一个, 彩铃, 歌手, 稀稀, 龙乐, 公司, 签约, ..."
50881,1,E:/vscode/file/trec06c/data/169/181,"[男生, 思路, 简单, 心痛, 直说, 原因, 不让, 担心, 他累, 不去, 撒娇, 撒..."
10843,0,E:/vscode/file/trec06c/data/036/043,[]
4689,0,E:/vscode/file/trec06c/data/015/189,"[本港, 会计师, 权威机构, 香港, 瑞丰, 会计师, 事务所, 注册, 海外, 国际, ..."


In [ ]:
from gensim.models import Word2Vec
from tqdm import tqdm_notebook

# 移除一些不必要的警告
import warnings

warnings.filterwarnings("ignore")

# 导入上面保存的分词数组
data = df["words"]

# 训练 Word2Vec 浅层神经网络模型
w2v_model = Word2Vec(vector_size=100, min_count=10)
w2v_model.build_vocab(data)
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=5)
w2v_model

In [25]:
def sum_vec(text):
    # 对每个句子的进行词向量求和计算
    vec = np.zeros(100).reshape((1, 100))
    for word in text:
        try:
            # 得到句子中每个词的词向量并累加在一起
            vec += w2v_model.wv.get_vector(word).reshape((1, 100))
        except KeyError:
            continue
    return vec

# 将词向量保存为 Ndarray
data_vec = np.concatenate([sum_vec(z) for z in tqdm_notebook(data)])
data_vec.shape

  0%|          | 0/10000 [00:00<?, ?it/s]

(10000, 100)

In [26]:
from sklearn.model_selection import train_test_split

feature_data = data_vec
label_data = df[0].values
# 分割数据
X_train, X_test, y_train, y_test = train_test_split(
    feature_data, label_data, test_size=0.2, random_state=4
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 100), (2000, 100), (8000,), (2000,))

In [27]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()  # 定义伯努利模型分类器
model.fit(X_train, y_train)  # 模型训练
y_pred = model.predict(X_test)  # 模型预测
y_pred

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [28]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.946